# Decorator
> 사전적 의미로는 '장식가' 또는 '인테리어 디자이너' 등의 의미를 가지고 있습니다. 이름 그대로, 자신의 방을 예쁜 벽지나 커튼으로 장식을 하듯이, 기존의 코드에 여러가지 기능을 추가하는 파이썬 구문이라고 생각하면 됩니다.

- https://schoolofweb.net/blog/posts/파이썬-데코레이터-decorator/
- https://engineer-mole.tistory.com/181

In [1]:
# python closure
def outer_func(msg):
  def inner_func():
    print(msg)

  return inner_func

hi = outer_func('hi')
bye = outer_func('byb')

hi()
bye()

hi
byb


### 1. 데코레이터

> 데코레이터 코드는 위의 클로저 코드와 비슷합니다. 다만 함수를 다른 함수의 인자로 전달한다는 점이 조금 다릅니다.   
> 아래는 가장 간단한 데코레이터 예제입니다.

In [2]:
def decorator_func(original_func):  # 1
  def wrapper_func(): # 5
    return original_func()  # 7
  
  return wrapper_func # 6

def display_func(): # 2
  print('display ~~~')  # 8

decorated_display = decorator_func(display_func)  # 3
decorated_display() # 4

display ~~~


> 복잡한 데코레이터라는 것을 도데체 왜 사용하는 걸까?   
> 기존의 코드를 수정하지 않고도, 래퍼(wrapper)함수를 이용하여 여러가지 기능을 추가할 수가 있기 때문이다.

In [4]:
def decorator_func(original_func):
  def wrapper_func():
    print(f'{original_func.__name__} 함수가 호출되기전 입니다.')
    return original_func()
  
  return wrapper_func 

def display_1():
  print('display_1 ~~~~')

def display_2():
  print('display_2 ~~~')

display1 = decorator_func(display_1)
display2 = decorator_func(display_2)

display1()
print('-'*10)
display2()

display_1 함수가 호출되기전 입니다.
display_1 ~~~~
----------
display_2 함수가 호출되기전 입니다.
display_2 ~~~


> 위의 코드를 데코레이터 함수가 적용된 코드로 수정

In [6]:
def decorator_func(original_func):
  def wrapper_func():
    print(f'{original_func.__name__} 함수가 호출되기전 입니다.')
    return original_func()
  
  return wrapper_func 

@decorator_func
def display_1():
  print('display_1 ~~~~')

@decorator_func
def display_2():
  print('display_2 ~~~')

# display1 = decorator_func(display_1)
# display2 = decorator_func(display_2)

display_1()
print('-'*10)
display_2()

display_1 함수가 호출되기전 입니다.
display_1 ~~~~
----------
display_2 함수가 호출되기전 입니다.
display_2 ~~~


### 2. 인수를 가진 함수를 데코레이팅
- *args: 임의의 수의 필수 파라미터를 수용함  
```python
# 예제
def one(*args):
  print(args)

one(1, 2, 3)

def add(x, y):
  return x + y

lst = [1, 2]
add(lst[0], lst[1]) #1
add(*lst) #1과 완전히 같음
```
- **kwargs: 리스트가 아닌 사전형 파라미터
```python
# 예제
def one(**kwargs):
  print(kwargs)

one(x=1, y=2)

def add(x, y):
  return x + y

dct = {'x':1, 'y':2}
add(**dct)
```

In [8]:
def decorator_func(original_func):
  def wrapper_func(*args, **kwargs):
    print(f'{original_func.__name__} 함수가 호출되기전 입니다.')
    return original_func(*args, **kwargs)
  
  return wrapper_func 

@decorator_func
def display_1():
  print('display_1 ~~~~')

@decorator_func
def display_2(name, age):
  print(f'{name}의 나이는 {age}')


display_1()
print('-'*10)
display_2('John', 25)

display_1 함수가 호출되기전 입니다.
display_1 ~~~~
----------
display_2 함수가 호출되기전 입니다.
John의 나이는 25


### 3. 데코레이터 클래스 버전

In [10]:
class DecoratorClass:
  def __init__(self, original_func):
    self.original_func = original_func

  def __call__(self, *args, **kwargs):
    print(f'{self.original_func.__name__} 함수가 호출되기전 입니다.')
    return self.original_func(*args, **kwargs)

@DecoratorClass
def display1():
  print('display1 ~~~~~')

@DecoratorClass
def display2(name, age):
  print(f'{name}의 나이는 {age}')

display_1()
print('-'*10)
display_2('John', 25)

display_1 함수가 호출되기전 입니다.
display_1 ~~~~
----------
display_2 함수가 호출되기전 입니다.
John의 나이는 25


### 4. 실전 예제

In [13]:
"""
리눅스나 유닉스 서버 관리자는 스크립트가 실행되는 시간을 측정하기 위한 기능
"""
import datetime
import time 

def my_logger(original_func):
  import logging 
  filename = f'{original_func.__name__}.log'
  logging.basicConfig(handlers=[logging.FileHandler(filename, 'a', 'utf-8')], level=logging.INFO) 

  def wrapper(*args, **kwargs):
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
    logging.info(f'[{timestamp}] 실행결과 args - {args}, kwargs - {kwargs}')
    print(f'[{timestamp}] 실행결과 args - {args}, kwargs - {kwargs}')
    return original_func(*args, **kwargs)

  return wrapper

@my_logger
def display_info(name, age):
  time.sleep(1)
  print(f'display_info ~~~~')

display_info('Cho', 40)

[2022-04-08 18:46] 실행결과 args - ('Cho', 40), kwargs - {}
display_info ~~~~


In [14]:
"""
프로그램이 실행되는 시간 측정
"""
import datetime 
import time 

def my_timer(original_func):
  import time 

  def wrapper(*args, **kwargs):
    t1 = time.time()
    result = original_func(*args, **kwargs)
    t2 = time.time() - t1
    print(f'{original_func.__name__} 함수의 실행된 총 시간: {t2}')
    return result 

  return wrapper 

@my_timer
def display_info(name, age):
  time.sleep(1)
  print(f'display_info ~~~~')

display_info('Cho', 40)

display_info ~~~~
display_info 함수의 실행된 총 시간: 1.0051250457763672


In [19]:
"""
데코레이터 두개 동시 실행
"""
from functools import wraps
import datetime
import time 

def my_logger(original_func):
  import logging 
  filename = f'{original_func.__name__}.log'
  logging.basicConfig(handlers=[logging.FileHandler(filename, 'a', 'utf-8')], level=logging.INFO) 

  @wraps(original_func)
  def wrapper(*args, **kwargs):
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
    logging.info(f'[{timestamp}] 실행결과 args - {args}, kwargs - {kwargs}')
    print(f'[{timestamp}] 실행결과 args - {args}, kwargs - {kwargs}')
    return original_func(*args, **kwargs)

  return wrapper

def my_timer(original_func):
  import time 

  @wraps(original_func)
  def wrapper(*args, **kwargs):
    t1 = time.time()
    result = original_func(*args, **kwargs)
    t2 = time.time() - t1
    print(f'{original_func.__name__} 함수의 실행된 총 시간: {t2}')
    return result 

  return wrapper   

@my_timer
@my_logger
def display_info(name, age):
  time.sleep(1)
  print(f'display_info ~~~~')

display_info('Cho', 40)

[2022-04-08 19:02] 실행결과 args - ('Cho', 40), kwargs - {}
display_info ~~~~
display_info 함수의 실행된 총 시간: 1.0044019222259521
